In [174]:
from pathlib import Path
import pandas as pd
import altair as alt
import seaborn as sns

In [175]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [176]:
data_dir = Path.cwd().parent.parent / "results" / "Case04-r04_10Tx-4Tk"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,1,True,80.509775,2,0.000000,MTV,Countdown
1,1,True,80.509775,3,30.028685,Tanker,Countdown
2,0,True,77.633622,2,6.000000,Tanker,Countdown
3,0,True,77.633622,3,66.028685,Tanker,Countdown
4,0,True,77.633622,4,70.009773,Tanker,Countdown


#### Summary Statistics

In [177]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.8868868868868869 = 88.689 %



#### Mission Duration

In [187]:
df = (
    mc_results[mc_results.outcome == True]
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "duration"]]
)

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[50, 200], step=10)),
    y='count()',
).properties(
    width = 600
)

alt.Chart(...)

Compute the Emprical CDF

In [188]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(x = "duration", y = "prob").properties(width=500, height=250)
    return fig


In [189]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [190]:
# fig = plot_ECDF(df)
# fig.interactive()

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [193]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [196]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [200]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,1,True,80.509775,2,0.000000,MTV,Countdown
1,1,True,80.509775,3,30.028685,Tanker,Countdown
2,0,True,77.633622,2,6.000000,Tanker,Countdown
3,0,True,77.633622,3,66.028685,Tanker,Countdown
4,0,True,77.633622,4,70.009773,Tanker,Countdown


In [201]:
mc_results.query("anomaly_count > 1")
# df = mc_results.query("replicant == 3")
# df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,1,True,80.509775,2,0.000000,MTV,Countdown
1,1,True,80.509775,3,30.028685,Tanker,Countdown
2,0,True,77.633622,2,6.000000,Tanker,Countdown
3,0,True,77.633622,3,66.028685,Tanker,Countdown
4,0,True,77.633622,4,70.009773,Tanker,Countdown
...,...,...,...,...,...,...,...
3762,994,True,148.814910,11,114.459183,Tanker,Countdown
3763,994,True,148.814910,12,115.849296,Tanker,Countdown
3764,994,True,148.814910,13,116.006062,Tanker,Countdown
3765,999,True,73.627742,2,19.000000,Tanker,Countdown


In [202]:
# df.anomaly_activity.value_counts()

In [203]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,0,Countdown,3
1,1,Countdown,2
2,2,Countdown,3
3,3,Countdown,4
4,4,Countdown,6
...,...,...,...
1195,994,Ascent,1
1196,995,Countdown,4
1197,996,Countdown,1
1198,998,Countdown,1


In [204]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [214]:
df = (
    mc_results
    # mc_results[mc_results.anomaly_activity != "Countdown"]
    .groupby("replicant", as_index=False)
    ["anomaly_activity"]
    .value_counts()
)

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)

In [215]:
df[df.anomaly_activity != "Countdown"]

,replicant,anomaly_activity,count
17,17,Ascent,1
19,18,Ascent,1
21,19,Ascent,1
31,28,Ascent,2
34,30,Ascent,1
...,...,...,...
1182,984,RPOD,1
1186,987,RPOD,1
1187,988,Ascent,1
1189,989,Orbit Insertion,1


In [216]:
mc_results

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,1,True,80.509775,2,0.000000,MTV,Countdown
1,1,True,80.509775,3,30.028685,Tanker,Countdown
2,0,True,77.633622,2,6.000000,Tanker,Countdown
3,0,True,77.633622,3,66.028685,Tanker,Countdown
4,0,True,77.633622,4,70.009773,Tanker,Countdown
...,...,...,...,...,...,...,...
3762,994,True,148.814910,11,114.459183,Tanker,Countdown
3763,994,True,148.814910,12,115.849296,Tanker,Countdown
3764,994,True,148.814910,13,116.006062,Tanker,Countdown
3765,999,True,73.627742,2,19.000000,Tanker,Countdown
